In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
dataset_root = 'datasets'

In [4]:
def resize_image(image, size=(32,32)):
    return cv2.resize(image, size)

In [5]:
def normalize_image(image):
    return image / 255.0

In [6]:
def one_hot_encode(labels):
    encoder = OneHotEncoder(sparse_output=False)
    labels = np.array(labels).reshape(-1, 1)
    encoder.fit(labels)
    return encoder.transform(labels)

In [15]:
def load_data(folder):
    images = []
    labels = []

    train_df = pd.read_csv(os.path.join(dataset_root, folder))

    for index, row in train_df.iterrows():
        image_file = row['Path']
        image_path = os.path.join(dataset_root, image_file)
        
        image = cv2.imread(image_path)
        image = resize_image(image)
        image = normalize_image(image)
        
        images.append(image)
        labels.append(row['ClassId'])
    
    images = np.array(images)
    labels = np.array(labels)
    
    labels = one_hot_encode(labels)
    
    return images, labels

In [19]:
images, labels = load_data('Train.csv')

print(images.shape)
print(labels.shape)

(39209, 32, 32, 3)
(39209, 43)


In [20]:
test_images, test_labels = load_data('Test.csv')

print(test_images.shape)
print(test_labels.shape)

(12630, 32, 32, 3)
(12630, 43)


In [23]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size = 0.2, random_state = 42)



X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(test_images, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(test_labels, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [24]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 43)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.reshape(-1, 128 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [27]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs = inputs.permute(0,3,1,2)
        outputs = model(inputs)
        loss = criterion(outputs, torch.argmax(labels, dim=1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.permute(0,3,1,2)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Validation Accuracy: {correct / total}")

Epoch [1/100], Loss: 1.9993639249058701
Epoch [2/100], Loss: 0.40622211727190405
Epoch [3/100], Loss: 0.15092717932419597
Epoch [4/100], Loss: 0.08763488618061402
Epoch [5/100], Loss: 0.049753508090502496
Epoch [6/100], Loss: 0.048499067420426326
Epoch [7/100], Loss: 0.031169001180432025
Epoch [8/100], Loss: 0.02300630310803316
Epoch [9/100], Loss: 0.02164749920133408
Epoch [10/100], Loss: 0.027598727248873904
Epoch [11/100], Loss: 0.014113223933718047
Epoch [12/100], Loss: 0.017942870086069467
Epoch [13/100], Loss: 0.016626567799655358
Epoch [14/100], Loss: 0.012516138163912477
Epoch [15/100], Loss: 0.001166876990053854
Epoch [16/100], Loss: 0.000307378234700502
Epoch [17/100], Loss: 0.033619004943893055
Epoch [18/100], Loss: 0.017150483663073434
Epoch [19/100], Loss: 0.00899309562340278
Epoch [20/100], Loss: 0.014467404398549637
Epoch [21/100], Loss: 0.019687804872371113
Epoch [22/100], Loss: 0.008352749909513912
Epoch [23/100], Loss: 0.006716178407711546
Epoch [24/100], Loss: 0.0130

In [28]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.permute(0,3,1,2)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f"Validation Accuracy: {correct / total}")

Validation Accuracy: 0.952652414885194
